In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.optimizers import Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import math
import secrets
import cv2
import h5py
import os
import shutil
import random
from PIL import Image

import warnings
warnings.filterwarnings("ignore")

# Create directories

In [2]:
parent_dir = os.getcwd()
directory = ['L1', 'L2', 'L1_augmented', 'L2_augmented']

for i in directory:
    path = os.path.join(parent_dir, i) 
    os.mkdir(path) 
    
path = os.path.join(parent_dir, 'COMP90086_2023_TLLdataset', 'train', 'left_augmented')
os.mkdir(path)

# Implement data augmentation
- Horizontal Flip
- Contrast
- Brightness

In [3]:
path = ".\COMP90086_2023_TLLdataset\\train\\left"
dir_list = os.listdir(path)
train_left_images_list = dir_list

# https://www.geeksforgeeks.org/how-to-flip-an-image-horizontally-or-vertically-in-python/
# https://www.tutorialspoint.com/how-to-change-the-contrast-and-brightness-of-an-image-using-opencv-in-python

for i in train_left_images_list:
    # open the original image
    original_img = Image.open(".\COMP90086_2023_TLLdataset\\train\\left\\{}".format(i))

    horz_img = original_img.transpose(method=Image.FLIP_LEFT_RIGHT)
    horz_img.save(".\COMP90086_2023_TLLdataset\\train\\left_augmented\\{}_1.jpg".format(i[:3]))
    
    # close all our files object
    original_img.close()
    horz_img.close()
    
    # read the input image
    image_ = cv2.imread(".\COMP90086_2023_TLLdataset\\train\\left\\{}".format(i))

    # define the alpha and beta
    alpha = 1.1 # Contrast control
    # call convertScaleAbs function
    adjusted = cv2.convertScaleAbs(image_, alpha=alpha)
    cv2.imwrite(".\COMP90086_2023_TLLdataset\\train\\left_augmented\\{}_2.jpg".format(i[:3]), adjusted)
    
    beta = -10 # Brightness control
    # call convertScaleAbs function
    adjusted2 = cv2.convertScaleAbs(image_, beta=beta)
    cv2.imwrite(".\COMP90086_2023_TLLdataset\\train\\left_augmented\\{}_3.jpg".format(i[:3]), adjusted2)

# Define feature extraction functions

In [4]:
def extract_features_VGG(img_path):
    # Load the VGG16 model pretrained on ImageNet data
    model = VGG16(include_top=False, weights='imagenet', pooling='max')

    img = image.load_img(img_path, target_size=(224, 224))
    array = image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    # Preprocess input
    array = preprocess_input(array)
    # Extract features
    features = model.predict(array)

    # Flatten the features to convert it to 1D array
    return features.flatten()

def extract_features_ResNet(img_path):
    # Load the ResNet50 model pretrained on ImageNet data
    model = ResNet50(include_top=False, weights='imagenet', pooling='max')
    
    img = image.load_img(img_path, target_size=(224, 224))
    array = image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    # Preprocess input
    array = preprocess_input(array)
    # Extract features
    features = model.predict(array)

    # Flatten the features to convert it to 1D array
    return features.flatten()

def extract_features_DenseNet(img_path):
    # Load the DenseNet121 model pretrained on ImageNet data
    model = DenseNet121(include_top=False, weights='imagenet', pooling='max')
    
    img = image.load_img(img_path, target_size=(224, 224))
    array = image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    # Preprocess input
    array = preprocess_input(array)
    # Extract features
    features = model.predict(array)

    # Flatten the features to convert it to 1D array
    return features.flatten()

In [5]:
path = ".\COMP90086_2023_TLLdataset\\train\\left_augmented\\"
dir_list = os.listdir(path)
train_left_images_list = dir_list

## VGG

In [6]:
left_features = []

for i in range(len(train_left_images_list)):
    img_path = '.\COMP90086_2023_TLLdataset\\train\\left_augmented\\{}'.format(train_left_images_list[i])
    left_features.append(extract_features_VGG(img_path))

In [7]:
np.save('AUG_vgg_train_left.npy', left_features)

## ResNet

In [9]:
left_features = []

for i in range(len(train_left_images_list)):
    img_path = '.\COMP90086_2023_TLLdataset\\train\\left_augmented\\{}'.format(train_left_images_list[i])
    left_features.append(extract_features_ResNet(img_path))

In [10]:
np.save('AUG_resnet_train_left.npy', left_features)

## DenseNet

In [11]:
left_features = []

for i in range(len(train_left_images_list)):
    img_path = '.\COMP90086_2023_TLLdataset\\train\\left_augmented\\{}'.format(train_left_images_list[i])
    left_features.append(extract_features_DenseNet(img_path))

In [12]:
np.save('AUG_densenet_train_left.npy', left_features)

# Create a new CSV file for training data

In [8]:
train_data = pd.read_csv(".\COMP90086_2023_TLLdataset\\train.csv")

path = ".\COMP90086_2023_TLLdataset\\train\\left_augmented"
dir_list = os.listdir(path)
train_left_images_list = dir_list

# Create two lists to store the corresponding filenames of left and right images 
left = []
right = []
for i in range(len(train_left_images_list)):
    if train_left_images_list[i][:3] == train_data['left'][i//3]:
        left.append(train_left_images_list[i][:5])
        right.append(train_data['right'][i//3])
        
augmented = pd.DataFrame({'left': left, 'right': right})
train_overall = pd.concat([train_data, augmented])
# Save to CSV
train_overall.to_csv('.\COMP90086_2023_TLLdataset\\train_augmented.csv', index=False)

# Copy the original left images to the "left_augmented" folder

In [9]:
# Copy the original left images to the "left_augmented" folder

path = ".\COMP90086_2023_TLLdataset\\train\\left"
dir_list = os.listdir(path)
train_left_images_list = dir_list

for i in train_left_images_list:
    src = os.path.join(parent_dir, 'COMP90086_2023_TLLdataset', 'train', 'left', i)
    dst = os.path.join(parent_dir, 'COMP90086_2023_TLLdataset', 'train', 'left_augmented')
    shutil.copy(src, dst)